In [1]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

# Imports

In [2]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [3]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path

In [4]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [26]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run ../../Functions/1.2/data_fct_Adrien_point_by_point

matplotlib.rcParams.update({'font.size': 21})

In [6]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [7]:
def load_Temp_init_bin_Lan(str_load, flag_print):      

    fid = open(str_load+'.bin',  'rb')   

    a    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  int32, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    N_ions = aux[0]   

    b    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  int32, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    j_save = aux[0]   

    c    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  float64, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    dt_j_save_next = aux[0]   

    d    = fromfile(fid,  int32, 1)        # Read record start tag   
    aux  = fromfile(fid,  float64, 1)   
    junk = fromfile(fid,  int32, 1)        # Read record stop tag   
    eta = aux[0]   

    e    = fromfile(fid,  int32,   1   )        # Read record start tag   
    Temp = fromfile(fid,  float64, 1)   
    junk = fromfile(fid,  int32,   1   )        # Read record stop tag   

    f    = fromfile(fid,  int32,   1   )        # Read record start tag 
    save_T = fromfile(fid,  float64, 7*j_save)   
    junk = fromfile(fid,  int32,   1   )        # Read record stop tag

    fid.close   
    
    print('len save_T', junk)   

    if flag_print == 1:
        print(a, b, c, d, e, f)   
        print('N_ions', N_ions)   
        print('j_save', j_save)   
        print('dt_j_save_next', dt_j_save_next)   
        print('eta', eta)   
        print('Temp', Temp)   


    save_T = reshape(save_T, (j_save, 7), order='F')   
    print(save_T[0])   

    return N_ions, j_save, dt_j_save_next, eta, Temp, save_T 

# Sélection fichier

In [8]:
## GUI for data loading
# Select one data file all the way down to the directories
# SELECT Temp_SimuTypeQ_N ... .dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try00
> Filename : Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [0, 5, 12, 21, 30, 39, 49]
> slashcond | 5
Points deleted because they were not complete []   0 pt(s)
Total number of data directories 868


In [9]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

['DC01_RF08', 'DC01_RF09', 'DC01_RF10', 'DC01_RF11', 'DC03_RF08', 'DC03_RF09', 'DC03_RF10', 'DC03_RF11', 'DC05_RF08', 'DC05_RF09', 'DC05_RF10', 'DC05_RF11', 'DC07_RF08', 'DC07_RF09', 'DC07_RF10', 'DC07_RF11', 'DC09_RF08', 'DC09_RF09', 'DC09_RF10', 'DC09_RF11', 'DC11_RF08', 'DC11_RF09', 'DC11_RF10', 'DC11_RF11', 'DC18_RF08', 'DC18_RF09', 'DC18_RF10', 'DC18_RF11']
000 > DC01_RF08
001 > DC01_RF09
002 > DC01_RF10
003 > DC01_RF11
004 > DC03_RF08
005 > DC03_RF09
006 > DC03_RF10
007 > DC03_RF11
008 > DC05_RF08
009 > DC05_RF09
010 > DC05_RF10
011 > DC05_RF11
012 > DC07_RF08
013 > DC07_RF09
014 > DC07_RF10
015 > DC07_RF11
016 > DC09_RF08
017 > DC09_RF09
018 > DC09_RF10
019 > DC09_RF11
020 > DC11_RF08
021 > DC11_RF09
022 > DC11_RF10
023 > DC11_RF11
024 > DC18_RF08
025 > DC18_RF09
026 > DC18_RF10
027 > DC18_RF11
> condition names ['DC', 'RF']
> number of points 28
> N_ions = 1024
> e_GMol = 50


In [10]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir,points_and_coord,condition_parameters,slash_cfg) # stopping_power

data0 = data_name[0]
Tvar = PMandT[1]

> Points | 28
> Simulations pour chaque point | ['Try00', 'Try01', 'Try02', 'Try03', 'Try04', 'Try05', 'Try06', 'Try07', 'Try08', 'Try09', 'Try10', 'Try11', 'Try12', 'Try13', 'Try14', 'Try15', 'Try16', 'Try17', 'Try18', 'Try19', 'Try20', 'Try21', 'Try22', 'Try23', 'Try24', 'Try25', 'Try26', 'Try27', 'Try28', 'Try29', 'Try30']
Hello
Point n° 0
00 - 00  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try00/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 01  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try01/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 02  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try02/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 03  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try03/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 04  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try04/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 05  >  /home/adrian/

02 - 01  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try01/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 02  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try02/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 03  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try03/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 04  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try04/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 05  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try05/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 06  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try06/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 07  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try07/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_S1.0RFG
02 - 08  >  /home/adrian/RemoteFS/Rivendel/20200903/DC01_RF10/Try08/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.1000D+01V_D1.0_

04 - 04  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF08/Try04/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.2000D+01V_D1.0_S1.0RFG
04 - 05  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF08/Try05/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.2000D+01V_D1.0_S1.0RFG
04 - 06  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF08/Try06/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.2000D+01V_D1.0_S1.0RFG
04 - 07  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF08/Try07/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.2000D+01V_D1.0_S1.0RFG
04 - 08  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF08/Try08/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.2000D+01V_D1.0_S1.0RFG
04 - 09  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF08/Try09/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.2000D+01V_D1.0_S1.0RFG
04 - 10  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF08/Try10/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.2000D+01V_D1.0_S1.0RFG
04 - 11  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF08/Try11/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.2000D+01V_D1.0_

06 - 07  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try07/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
06 - 08  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try08/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
06 - 09  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try09/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
06 - 10  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try10/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
06 - 11  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try11/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
06 - 12  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try12/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
06 - 13  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try13/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG
06 - 14  >  /home/adrian/RemoteFS/Rivendel/20200903/DC03_RF10/Try14/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.2000D+01V_D1.0_

08 - 10  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF08/Try10/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.3000D+01V_D1.0_S1.0RFG
08 - 11  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF08/Try11/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.3000D+01V_D1.0_S1.0RFG
08 - 12  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF08/Try12/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.3000D+01V_D1.0_S1.0RFG
08 - 13  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF08/Try13/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.3000D+01V_D1.0_S1.0RFG
08 - 14  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF08/Try14/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.3000D+01V_D1.0_S1.0RFG
08 - 15  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF08/Try15/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.3000D+01V_D1.0_S1.0RFG
08 - 16  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF08/Try16/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.3000D+01V_D1.0_S1.0RFG
08 - 17  >  /home/adrian/RemoteFS/Rivendel/20200903/DC05_RF08/Try17/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.3000D+01V_D1.0_

...

18 - 25  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF10/Try25/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
18 - 26  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF10/Try26/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
18 - 27  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF10/Try27/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
18 - 28  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF10/Try28/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
18 - 29  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF10/Try29/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
18 - 30  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF10/Try30/Temp_SimuTypeQ_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
Point n° 19
19 - 00  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF11/Try00/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG
19 - 01  >  /home/adrian/RemoteFS/Rivendel/20200903/DC09_RF11/Try01/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.500

20 - 28  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF08/Try28/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.7000D+01V_D1.0_S1.0RFG
20 - 29  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF08/Try29/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.7000D+01V_D1.0_S1.0RFG
20 - 30  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF08/Try30/Temp_SimuTypeQ_N01024_Vrf0053_Udc0.7000D+01V_D1.0_S1.0RFG
Point n° 21
21 - 00  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF09/Try00/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.7000D+01V_D1.0_S1.0RFG
21 - 01  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF09/Try01/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.7000D+01V_D1.0_S1.0RFG
21 - 02  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF09/Try02/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.7000D+01V_D1.0_S1.0RFG
21 - 03  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF09/Try03/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.7000D+01V_D1.0_S1.0RFG
21 - 04  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF09/Try04/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.700

Point n° 23
23 - 00  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF11/Try00/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.7000D+01V_D1.0_S1.0RFG
23 - 01  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF11/Try01/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.7000D+01V_D1.0_S1.0RFG
23 - 02  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF11/Try02/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.7000D+01V_D1.0_S1.0RFG
23 - 03  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF11/Try03/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.7000D+01V_D1.0_S1.0RFG
23 - 04  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF11/Try04/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.7000D+01V_D1.0_S1.0RFG
23 - 05  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF11/Try05/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.7000D+01V_D1.0_S1.0RFG
23 - 06  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF11/Try06/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.7000D+01V_D1.0_S1.0RFG
23 - 07  >  /home/adrian/RemoteFS/Rivendel/20200903/DC11_RF11/Try07/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.700

25 - 03  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try03/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_S1.0RFG
25 - 04  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try04/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_S1.0RFG
25 - 05  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try05/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_S1.0RFG
25 - 06  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try06/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_S1.0RFG
25 - 07  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try07/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_S1.0RFG
25 - 08  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try08/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_S1.0RFG
25 - 09  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try09/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_S1.0RFG
25 - 10  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF09/Try10/Temp_SimuTypeQ_N01024_Vrf0059_Udc0.6000D+01V_D1.0_

27 - 06  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try06/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 07  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try07/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 08  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try08/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 09  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try09/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 10  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try10/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 11  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try11/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 12  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try12/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_S1.0RFG
27 - 13  >  /home/adrian/RemoteFS/Rivendel/20200903/DC18_RF11/Try13/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.6000D+01V_D1.0_

# Plots
Nuage d'ions, Temp et Pos

## Load temperature from Langevin files

In [11]:
## Data loading with hardcode names
eta = 1.5e-20

plot_point = 0
plot_try = 0
# str_eta_chiffre    = float(f'{eta[plot_point]:1.2e}'[:4])/10
# str_eta_puissance  = int(re.sub('e','D',f'{eta[plot_point]:1.2e}'[6:]))-1
# str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
str_eta = '0.15D-19'
dirname_hc = str(data0[plot_point][plot_try])+'/'
# Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
data0_hc    = '/Temp_SimuTypeQ_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data4_hc    = '/Temp_SimuType4_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'
data2_hc    = '/Temp_SimuType2_01_N01024_Vrf0070_Udc0.5000D+01V_D1.0_S1.0RFG'

print(dirname_hc)
print(datainit_hc)

/home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try00/
Temp_3D_Harmo_N1024_T500uK_F0.15D-19Kg_s_5


In [12]:
N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T =\
load_Temp_init_bin_Lan(dirname_hc+datainit_hc,1)

len save_T [55440]
[4] [4] [8] [8] [8] [55440]
N_ions 1024
j_save 990
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.0005]
[5.05000000e-07 1.41358910e-05 2.19387267e-03 1.11238728e-05
 4.52098528e+01 1.51182185e+00 7.29968464e+01]


In [13]:
# Get T and fluo PM  All SimuType
onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and "SimuType" in f and ".dat" in f]
print(sort(onlyfiles))
only_init = [f for f in onlyfiles if "SimuTypeQ" in f]
ta, te, ti = plot_T_and_PM_InitQ_Inje_Evol(dirname_hc,only_init[0].strip('.dat')[4:],flag_plot=1,fig_name='0',xlim1=(0,11),ylim1=(2e-4,500),ylim2=(2,65))
# tt2, T_CM2, T_aux2, PM2
# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

['Temp_SimuType2_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuType4_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG50eV.dat'
 'Temp_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.dat'
 'xva_SimuType4_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG50eV.dat']


In [14]:
# recover duration for each step of simu (i_free__fly, i_laser_fly ...)

address = all_subdir[plot_point]
print(address)
info_files = sort( [f for f in listdir(address) if isfile(join(address, f)) and ".info" in f] )
print(sort(info_files))


tmp_info = loadtxt(data0[plot_point][0]+'/'+info_files[0],encoding ='utf-8',
            skiprows=1, usecols=(0),
            unpack=True)
eta = tmp_info[-2]
i_free__fly_Lan = tmp_info[-1]

tmp_info = loadtxt(data0[0][0]+'/'+'xva_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.info',encoding ='utf-8',
            skiprows=0, usecols=(0),
            unpack=True)
i_laser_fly_quench = tmp_info[-2]
i_laser_fly = tmp_info[-1]

n_dt = 100
dt = 1/2e6

print(eta)
print(i_free__fly_Lan)
print(i_laser_fly_quench)
print(i_laser_fly)

/home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try00
['Langevin_cooling.info'
 'xva_SimuType2_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG50eV.info'
 'xva_SimuType4_01_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG50eV.info'
 'xva_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.info']
1.5e-20
1000.0
8000.0
6000.0


In [15]:
# Plot temperature Langevin + md laser  ALL SimuType

tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
tt_md = ta[0]
T_aux = te[0]

tt = concatenate((tt_Lan,tt_md))
T_all = concatenate((save_T[:,4:],T_aux),axis=0)

PM = ti[0]

#fig_name = file_name[-9:] 
figure('Temperature'); clf()
ax1 = subplot(211)
ax1.semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
ax1.semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
ax1.semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
#semilogy(tt_Lan*1.e3,save_T[:,0], label='Tx',color='C0')
#semilogy(tt_Lan*1.e3,save_T[:,1], label='Ty',color='C1')
#semilogy(tt_Lan*1.e3,save_T[:,2], label='Tz',color='C2')
ax1.vlines(ta[1]*1e3,1e-4,1e4,color='red')
ax1.vlines(ta[2]*1e3,1e-4,1e4,color='red')
time_switch = i_free__fly_Lan*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='green')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
ax1.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
ax1.grid()
# annotate('Laser ON', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul')
,
# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

ax2 = subplot(212,sharex=ax1)
plot(tt_md*1e3,PM, label='Tx')
time_switch = i_free__fly_Lan*dt
ax2.vlines(time_switch*1e3,1e-4,90,color='blue')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax2.vlines(time_switch*1e3,1e-4,90,color='green')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax2.vlines(time_switch*1e3,1e-4,90,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
ax2.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,90,color='purple')
ax2.grid()
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

xlabel('time[ms]')
ylabel('Counts')

plt.tight_layout()
subplots_adjust(hspace=0.015)

In [ ]:
# Plot all temp for all conditions
fname = 'Temp_allpoints'


fig, ax = plt.subplots(5, 4,num=fname,clear=True,sharex=True,sharey=True)  # (a,b)
fig.set_size_inches(11.69,8.27)
# suptitle('Saturation 866')

to_plot = 0

for l in range(0,4): # b
    for k in range(0,5): # a
        plot_point = to_plot
#             str_eta_chiffre    = float(f'{eta[patate]:1.2e}'[:4])/10
#             str_eta_puissance  = int(re.sub('e','D',f'{eta[patate]:1.2e}'[6:]))-1
#             str_eta = f'{str_eta_chiffre:1.2f}'+'D-'+str(str_eta_puissance)
        str_eta = '0.15D-19'
        dirname_hc = str(data0[plot_point][plot_try])+'/'
        # Temp_3D_Harmo_N1024_T500uK_F0.20D-20Kg_s_5
        datainit_hc = 'Temp_3D_Harmo_N1024_T500uK_F'+str(str_eta)+'Kg_s_5'
        N_ions, j_save, dt_j_save_next, eta_float, Temp, save_T = \
        load_Temp_init_bin_Lan(dirname_hc+datainit_hc,flag_print=0)
        # Get T and fluo PM
        onlyfiles = [f for f in listdir(dirname_hc) if isfile(join(dirname_hc, f)) and 'SimuType' in f and '.dat' in f]
        ta, te, ti = plot_T_and_PM_InitQ_Inje_Evol(dirname_hc,sort(onlyfiles)[2].strip('.dat')[4:],flag_plot = 0,fig_name = 'Temp_allpoints')

        tt_Lan = linspace(0,dt_j_save_next*j_save,j_save)
        tt_md = ta[0]
        T_aux = te[0]
        
#         tt_Lan = linspace(0,dt_j_save_next*j_save,j_save) # dt_j_save_next*100 duration of Langevin run  ,
#         tt_md = ta[0]
#         T_aux = te[0]
#         tt = concatenate((tt_Lan,tt_md))
#         T_all = concatenate((save_T[:,4:],T_aux),axis=0)

        tt = concatenate((tt_Lan,tt_md))
        T_all = concatenate((save_T[:,4:],T_aux),axis=0)

        ax[k,l].semilogy(tt*1e3,T_all[:,0], label='Tx',color='C0')
        ax[k,l].semilogy(tt*1e3,T_all[:,1], label='Ty',color='C1')
        ax[k,l].semilogy(tt*1e3,T_all[:,2], label='Tz',color='C2')
#         ax[k,l].vlines(ta[1]*1e3,1e-4,1e4,color='red')
#         ax[k,l].vlines(ta[2]*1e3,1e-4,1e4,color='red')
# print(i_free__fly_Lan)
# print(i_laser_fly_quench)
# print(i_laser_fly)
        time_switch = i_free__fly_Lan*dt
        ax[k,l].vlines(time_switch*1e3,1e-4,1e4,color='blue')
        time_switch = (i_free__fly_Lan+i_laser_fly)*dt
        ax[k,l].vlines(time_switch*1e3,1e-4,1e4,color='green')
        time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
        ax[k,l].vlines(time_switch*1e3,1e-4,1e4,color='orange')
        time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
        ax[k,l].vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
        ax[k,l].set_yticks([1e-4,1e-2,1,1e2,1e4])
        ax[k,l].grid()
        #     ax[k].set_ylim(-0.1,0.55)
        ax[k,l].xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=2))
        ax[k,l].annotate(str(list(dict.fromkeys(points_and_coord))[plot_point]), xy=(0.20, 0.99), xycoords='axes fraction', # pts_concerned[to_plot]
        size=12, ha='left', va='top',
        bbox=dict(boxstyle='round', fc='w'))
        to_plot += 1
        if to_plot == 19:
            break
    if to_plot == 19:
        break

# ax[3].set_xlim(-10,4000)
# ax[k,l].set_ylim(2e-4,1e4)
fig.suptitle(str(eta) + 'Kg.s-1')
fig.text(0.56, 0.04, 'time [ms]', ha='center')
fig.text(0.04, 0.54, 'T [K]', va='center', rotation='vertical')

# fig.suptitle(pt_to_plot[0][:-6],fontsize=10)

# fig.text(0.5, 0.01, 'f', ha='center')
# fig.text(0.01, 0.5, r'$| FFT(HC(t)) | / N_{sample}$', va='center', rotation='vertical')

fig.set_size_inches(11.69,8.27)
plt.tight_layout()
subplots_adjust(hspace=0.05, wspace = 0.05, bottom = 0.120, left = 0.140, top = 0.92) # hspace=0, top=0.95, left = 0.065, bottom = 0.07
savefig(fname+'.png') 

In [16]:
Gmol_data[2]

detect_rate = []
detect_threshold = 1

for k in range(shape(Gmol_data[2])[0]):
    print('k',k)
    detect_rate.append(0)
    for l,m in enumerate(Gmol_data[2][k]):
        print('... l',l)
        if m > detect_threshold:
            detect_rate[k] += 1
    detect_rate[k] = detect_rate[k] / shape(Gmol_data[2])[1]
    
for n,o in enumerate(list(dict.fromkeys(points_and_coord))):
    print(o)
    print(detect_rate[n])

k 0
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
... l 6
... l 7
... l 8
... l 9
... l 10
... l 11
... l 12
... l 13
... l 14
... l 15
... l 16
... l 17
... l 18
... l 19
... l 20
... l 21
... l 22
... l 23
... l 24
... l 25
... l 26
... l 27
... l 28
... l 29
... l 30
k 1
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
... l 6
... l 7
... l 8
... l 9
... l 10
... l 11
... l 12
... l 13
... l 14
... l 15
... l 16
... l 17
... l 18
... l 19
... l 20
... l 21
... l 22
... l 23
... l 24
... l 25
... l 26
... l 27
... l 28
... l 29
... l 30
k 2
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
... l 6
... l 7
... l 8
... l 9
... l 10
... l 11
... l 12
... l 13
... l 14
... l 15
... l 16
... l 17
... l 18
... l 19
... l 20
... l 21
... l 22
... l 23
... l 24
... l 25
... l 26
... l 27
... l 28
... l 29
... l 30
k 3
... l 0
... l 1
... l 2
... l 3
... l 4
... l 5
... l 6
... l 7
... l 8
... l 9
... l 10
... l 11
... l 12
... l 13
... l 14
... l 15
... l 16
... l 17
... l 18
... l 19
... l 2

In [23]:
# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]
print(Detuning)

1.0


In [17]:
Udc = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 7.5, 3.33, 3.66, 4.5, 5.33, 5.66, 6, 6.5]
Urf = [10.77, 16.15, 21.54, 26.92, 32.31, 37.69, 43.08, 48.46, 53.85, 59.23, 64.61,
       70.00, 75.38, 80.77, 61.02, 62.82, 66.41, 68.20, 71.79, 55.64, 57.43]

In [29]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

In [87]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(Udc[int(points_and_coord[h][0])])
    y.append(Urf[int(points_and_coord[h][1])])
    z.append(detect_rate[g])
    
fname = '2D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][:-15],fontsize=25)

im = ax.scatter(x,y,
        marker='D',s=400,edgecolor='black',zorder = 1000,
        c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))
ax.set_xlabel(r'$U_{DC}$ [V]')
ax.set_ylabel(r'$U_{RF}$ [V]')
ax.grid()

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=13,
                ha='center',va='center',weight='bold',zorder = 1000)
    # ax.annotate(str(txt), (x[i], y[i]-0.70),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'.png',dpi=300)

In [53]:
np.savez('20200903_Rivendel',Condition = list(dict.fromkeys(points_and_coord)),Udc=Udc,Urf=Urf,SNR=Gmol_data[2])
print(shape(Gmol_data[2]))
babebibobu = np.load('/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint/2.1/20200903_Rivendel.npz')
babebibobu['SNR'][0]

(28, 31)


In [164]:
address = all_subdir[14*31]
# dirname_hc = str(data0[1][0])+'/'
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and 'SimuType' in f and '.dat' in f]
my_file = '{}/xva{}'.format(address,sort(onlyfiles)[2].strip('.dat')[4:])
plot_XYZ(my_file,fig_name='2',fig_title='XYZ')

In [191]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(mean(dim_nu[g,:,0])/mean(dim_nu[g,:,2]))
    y.append(mean(dim_nu[g,:,2])*1000)
    z.append(detect_rate[g])
    
    
fname = '2D_GMol_var_fluo_contour_Lvs_RoverL'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][:-15],fontsize=25)

im = ax.scatter(x,y,
        marker='D',s=250,edgecolor='black',zorder = 1000,
        c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))
ax.set_xlabel(r'mean R/L')
ax.set_ylabel(r'mean L [µm]')

for i, txt in enumerate(points_and_coord):
    ax.annotate(str(i), (x[i], y[i]),color='xkcd:black',fontsize=10,
                ha='center',va='center',weight='bold',zorder = 1000)
    # ax.annotate(str(txt), (x[i], y[i]-0.70),fontsize=9)
#    if mean(cool_cloud[i])*13 < 13:
#        ax.annotate(f'{mean(cool_cloud[i])*13:.0f}',(x[i]-0.12, y[i]+0.25),fontsize=13,color='red',ha='right')
    #ax.annotate(str(txt)+'\n'+str(i), (x[i], y[i]))

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.05))
ax.minorticks_on()
# ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.grid(which='major', linestyle='-', linewidth='1', color='k')
ax.grid(b=True, which='minor', color='k', linestyle=':')
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'.png',dpi=300)

# SNR détaillé

In [123]:
plt.close('all')

In [122]:
# Recover Ca pos, speed and acc at the end of one specific step
# address = simutype6address[20]

my_plot = 1
for my_plot,ll in enumerate(list(dict.fromkeys(points_and_coord))):
    
    print(my_plot,ll)
    my_base = 1

    fname = 'SNR_'+'#'+str(my_plot)+'_'+ll
    fig = figure(fname); clf()
    fig.suptitle(r'SNR '+ll,color='xkcd:black',fontsize = 18)
    ax1 = subplot(111)
    my_bins = arange(-0.0,5.25,0.1)
    ax1.hist(Gmol_data[2][my_plot],bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
    ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
    ax1.grid()
    xlabel('SNR')

    fig.set_size_inches(11.69, 8.27)
    plt.tight_layout()
    subplots_adjust(hspace=0.50, top = 0.93)
    savefig(fname+'.png') 

0 DC01_RF08
1 DC01_RF09
2 DC01_RF10
3 DC01_RF11
4 DC03_RF08
5 DC03_RF09
6 DC03_RF10
7 DC03_RF11
8 DC05_RF08
9 DC05_RF09
10 DC05_RF10
11 DC05_RF11
12 DC07_RF08
13 DC07_RF09
14 DC07_RF10
15 DC07_RF11
16 DC09_RF08
17 DC09_RF09
18 DC09_RF10
19 DC09_RF11
20 DC11_RF08
21 DC11_RF09
22 DC11_RF10
23 DC11_RF11
24 DC18_RF08
25 DC18_RF09
26 DC18_RF10
27 DC18_RF11


# Distribution des vitesses

In [162]:
address = all_subdir[22]
# dirname_hc = str(data0[1][0])+'/'
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and 'SimuType' in f and '.dat' in f]
my_file = '{}/xva{}'.format(address,sort(onlyfiles)[2].strip('.dat')[4:])
print(my_file)
r_LC,v_LC,a_LC,v_LC_avg = load_xyz_init_bin_DP(my_file)  

/home/adrian/RemoteFS/Rivendel/20200903/DC01_RF08/Try22/xva_SimuTypeQ_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG


In [106]:
# Recover Ca pos, speed and acc at the end of one specific step
# address = simutype6address[20]
print(address)
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and "xva" in f and 'SimuTypeQ' in f and 'bin' in f]
r_LC, v_LC, a_LC, v_rf_avg = load_xyz_init_bin_DP(address+'/'+onlyfiles[0].strip('.bin'))


v_LC_x = v_LC[0]
v_LC_y = v_LC[1]
v_LC_z = v_LC[2]

my_base = 50

fig = figure('Terminal speed'); clf()
fig.suptitle(r'Atomic speed at the end of SimuType0 - RF - '+str(address[31:]),color='xkcd:black',fontsize = 18)
ax1 = subplot(311)
my_bins = arange(-250,251,20)
ax1.hist(v_LC_x,bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax1.grid()
xlabel('$v_x$ [m/s]')

ax2 = subplot(312)
ax2.hist(v_LC_y,bins=my_bins ,color='C1', edgecolor='black', linewidth=1.2)
ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax2.grid()
xlabel('$v_y$ [m/s]')

my_base = 0.2
ax3 = subplot(313)
my_bins = arange(-2,2.1,0.1)
# my_bins = arange(-30000,30001,5000)
ax3.hist(v_LC_z,bins=my_bins ,color='C2', edgecolor='black', linewidth=1.2)
ax3.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax3.grid()
xlabel('$v_z$ [m/s]')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)

/home/adrian/RemoteFS/Hobitton/20200903/DC01_RF11/Try04


In [107]:
# Recover Ca pos, speed and acc at the end of one specific step
# address = simutype6address[20]PleasePl
print(address)
onlyfiles = [f for f in listdir(address) if isfile(join(address, f)) and "xva" in f and 'SimuTypeQ' in f and 'bin' in f]
r_LC, v_LC, a_LC, v_rf_avg = load_xyz_init_bin_DP(address+'/'+onlyfiles[0].strip('.bin'))
print(address+'/'+onlyfiles[0])


v_LC_avg_x = v_rf_avg[0]/100
v_LC_avg_y = v_rf_avg[1]/100
v_LC_avg_z = v_rf_avg[2]/100

my_base = 5

fig = figure('Terminal speed 2'); clf()
fig.suptitle(r'Atomic speed at the end of SimuType0 - no RF - '+str(address[31:]),color='xkcd:black',fontsize = 18)
ax1 = subplot(311)
my_bins = arange(-25,26,2)
ax1.hist(v_LC_avg_x,bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax1.grid()
xlabel('$v_x$ [m/s]')

ax2 = subplot(312)
ax2.hist(v_LC_avg_y,bins=my_bins ,color='C1', edgecolor='black', linewidth=1.2)
ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax2.grid()
xlabel('$v_y$ [m/s]')

my_base = 0.2
ax3 = subplot(313)
my_bins = arange(-2,2.1,0.1)
# my_bins = arange(-30000,30001,5000)
ax3.hist(v_LC_avg_z,bins=my_bins ,color='C2', edgecolor='black', linewidth=1.2)
ax3.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax3.grid()
xlabel('$v_z$ [m/s]')

fig.set_size_inches(11.69, 8.27)
plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)

/home/adrian/RemoteFS/Hobitton/20200903/DC01_RF11/Try04
/home/adrian/RemoteFS/Hobitton/20200903/DC01_RF11/Try04/xva_SimuTypeQ_N01024_Vrf0070_Udc0.1000D+01V_D1.0_S1.0RFG.bin


In [93]:
m_kb_x_inv_n_ions  = m_Ca/(kb*N_ions*n_dt**2   )
T_aux_LC_x = sum(v_rf_avg[0]**2)*m_kb_x_inv_n_ions
T_aux_LC_y = sum(v_rf_avg[1]**2)*m_kb_x_inv_n_ions
T_aux_LC_z = sum(v_rf_avg[2]**2)*m_kb_x_inv_n_ions

fig = figure('Calculated temp'); clf()
fig.suptitle(r'Atomic speed at the end of SimuType0 - no RF - '+str(address[31:]),color='xkcd:black',fontsize = 18)

my_base = 1
my_bins = arange(0,1,0.005)

ax1 = subplot(311)
ax1.hist(T_aux_LC_x,bins=my_bins ,color='C0', edgecolor='black', linewidth=1.2)
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax1.grid()
xlabel('$v_x$ [m/s]')

ax2 = subplot(312)
ax2.hist(T_aux_LC_y,bins=my_bins ,color='C1', edgecolor='black', linewidth=1.2)
ax2.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax2.grid()
xlabel('$v_y$ [m/s]')

ax3 = subplot(313)
# my_bins = arange(-2,2.1,0.1)
# my_bins = arange(-30000,30001,5000)
ax3.hist(T_aux_LC_z,bins=my_bins ,color='C2', edgecolor='black', linewidth=1.2)
ax3.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=my_base))
ax3.grid()
xlabel('$v_z$ [m/s]')

fig.set_size_inches(11.69, 8.27)
# plt.tight_layout()
subplots_adjust(hspace=0.50, top = 0.93)

In [132]:
n_threads = 32
space = N_ions/n_threads
linspace(1,1024-space+1,n_threads,dtype='int'),linspace(space,1024,n_threads,dtype='int')

(array([  1,  33,  65,  97, 129, 161, 193, 225, 257, 289, 321, 353, 385,
        417, 449, 481, 513, 545, 577, 609, 641, 673, 705, 737, 769, 801,
        833, 865, 897, 929, 961, 993]),
 array([  32,   64,   96,  128,  160,  192,  224,  256,  288,  320,  352,
         384,  416,  448,  480,  512,  544,  576,  608,  640,  672,  704,
         736,  768,  800,  832,  864,  896,  928,  960,  992, 1024]))